### **Notebook 05 - Decision tree models**

### Introduction

In the previous notebook, I optimized and fitted two linear regression models, and evaluated how they do on the train and test sets. In this section, I'll explore decision tree-based models to see if they improve prediction accuracy of future AQI values. I'll compare these model(s) to the ones already built in Notebook 3 and 4. For this, I'll use the dataset with the lag features from the previous notebook as opposed to the original dataset.

A brief summary is provided at the end of the notebook.

**Table of content:**


- Loading data and importing libraries

- Data preprocessing:

    - Train-test split

- Modeling:

    - Decision-tree model
    - XGBoost
        - Hyperparameter optimization for XGBoost
- Summary

***
#### Loading data and importing libraries

In [1]:
# Importing libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from statsmodels.api import tsa
import statsmodels.api as sm

In [2]:
# Loading dataset

aqi_df = pd.read_csv('data/air_quality_with_lags.csv', parse_dates=['Date'], index_col='Date')
aqi_df.sample(3)

,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_name,lat,lng,population,density,...,Year,Month,Day,ma_last_7d,ma_last_30d,ma_last_quarter,AQI_lag_1days,AQI_lag_7days,AQI_lag_30days,AQI_lag_365days
Date,,,,,,,,,,,,,,,,,,,,,
1987-06-11,240,Very Unhealthy,Ozone,21,Los Angeles,California,34.1141,-118.4068,12531334.0,3267.0,...,1987,6,11,200.714286,165.466667,153.428571,232.0,232.0,245.0,248.0
1984-03-01,164,Unhealthy,Ozone,20,Los Angeles,California,34.1141,-118.4068,12531334.0,3267.0,...,1984,3,1,112.428571,117.833333,131.439560,115.0,110.0,140.0,57.0
1984-09-12,90,Moderate,Ozone,22,Los Angeles,California,34.1141,-118.4068,12531334.0,3267.0,...,1984,9,12,169.857143,213.833333,229.000000,67.0,255.0,237.0,297.0


***
#### Pre-processing

Since I already did some feature-engineering for the linear models and loaded the dataset that includes these lag and time features, I won't need to perform further feature engineering for the notebook. Before modeling, I'll scale and split the dataset into remainder and test sets. This time I won't be splitting it down any further, since I'll use cross-validation to optimize the models.

In [3]:
# Splitting X and y variables

X = aqi_df.drop(columns=['AQI', 'Category', 'Defining Parameter', 'Number of Sites Reporting', 'city_ascii', 'state_name', 'lat', 'lng', 'population', 'density', 'timezone'])
y = aqi_df['AQI']

#### Train-test split

In [4]:
# Train-test split into test and remainder sets to set aside some unseen data

X_remainder = pd.DataFrame(X.loc[X.index <= '2013-01-01'])
y_remainder = pd.DataFrame(y.loc[y.index <= '2013-01-01'])

X_test = pd.DataFrame(X.loc[X.index > '2013-01-01'])
y_test = pd.DataFrame(y.loc[y.index > '2013-01-01'])

#### Scaling the data

In [5]:
# Scaling the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_remainder_s = scaler.fit_transform(X_remainder)
X_test_s = scaler.transform(X_test)

X_remainder = pd.DataFrame(data=X_remainder_s, columns=X_remainder.columns, index=X_remainder.index)
X_test = pd.DataFrame(data=X_test_s, columns=X_test.columns, index=X_test.index)

#### Decision tree model

Reminder on the results of the baseline model:

| Model  |  Train results |  Test results |
|---|---|---|
|  Baseline (mean prediction) | 99.39%  | 100.31%  |

In [6]:
# Decision tree model

from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=1)
tree.fit(X_remainder, y_remainder);

In [7]:
preds_train = tree.predict(X_remainder).reshape(11690,1)
preds_test = tree.predict(X_test).reshape(3286,1)

In [8]:
mape_train = round(np.mean(abs((y_remainder - preds_train) / y_remainder)) * 100, 2)
mape_test = round(np.mean(abs((y_test - preds_test) / y_test)) * 100, 2)

print(mape_train)
print(mape_test)

36.52
44.16


We can see that the model is overfitting, it's error percentage is much lower on the training dataset than on the test set.

Let's add these results to the table.

| Model  |  Train results |  Test results |
|---|---|---|
|  Baseline (mean prediction) | 99.39%  | 100.31%  |
|  Decision Tree | 36.52%  | 44.16%  |

We can see that the model does much better than the baseline model, however, the error percentage is still quite big. Instead of further optimizing the decision tree model, I'll move on to build a more robust decision tree-based model. 

***
### XGBoost

First let's see how the model does with the default parameters without modifying or optimizing any of them.

In [186]:
# Building the first XGBoost model without hyperparameter optimization

xgb = XGBRegressor()
xgb.fit(X_remainder, y_remainder)

# Predictions on the training and test sets

preds_xgb_train = xgb.predict(X_remainder).reshape(11690,1)
preds_xgb_test = xgb.predict(X_test).reshape(3286,1)

In [187]:
# Extracting MAPE scores

mape_train = round(np.mean(abs((y_remainder - preds_xgb_train) / y_remainder)) * 100, 2)
mape_test = round(np.mean(abs((y_test - preds_xgb_test) / y_test)) * 100, 2)

print(mape_train)
print(mape_test)

15.49
27.33


The results are generally much better compared to the decision tree model we've just seen. The XGBoost model does a great job on predicting the target variable on the training set, however, it does a less decent job on the test set. It's heavily overfitting and the error % is still quite high on the test set.

| Model  |  Train results |  Test results |
|---|---|---|
|  Baseline (mean prediction) | 99.39%  | 100.31%  |
|  Decision Tree (max depth = 1) | 36.52%  | 44.16%  |
|  XGBoost() | 15.49%  | 27.33%  |

To improve these results, I'll perform a grid-search using 3-fold validation. I'll optimize the following parameters:

- `eta` (or learning rate): the default values is 0.3, but the previous model was heavily overfitting, so I'll try something smaller
- `max_depth`: adding more depth to the model makes is more complex, and more lilely to overfit, so I'll try something in the lower ranges first
- `n_estimators`: controls the number of trees in the model, increasing this value might lead to overfitting, so I'll try something between 500-700

Source: https://medium.com/@rithpansanga/the-main-parameters-in-xgboost-and-their-effects-on-model-performance-4f9833cac7c 

#### Hyperparameter optimization for XGBoost

In [11]:
# Grid search for XGBoost

from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

xgb = XGBRegressor(eval_metric='mape')

max_depth = [6, 7, 8]
estimators = [500, 600, 700]
rate = [0.01, 0.1, 0.2]

my_param_grid = ({'max_depth': max_depth,
                 'n_estimators': estimators,
                 'eta': rate})

# Cross-validation split for time-series data

my_cv = TimeSeriesSplit(n_splits=3).split(X_remainder)

In [12]:
# Instantiate the xgboost grid search

gridsearch = GridSearchCV(xgb, param_grid=my_param_grid, cv=my_cv, verbose=1)

# Fit the grid search

fitted_xgb = gridsearch.fit(X_remainder, y_remainder)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


In [202]:
# Best parameters

fitted_xgb.best_params_

{'eta': 0.01, 'max_depth': 6, 'n_estimators': 500}

We can see that the best parameters were all the smallest possible values I added earlier, so it's worth running another grid search with smaller values - this is to see if smaller values for the parameters improve the performance or not.

In [19]:
# Grid search for XGBoost

from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

xgb = XGBRegressor()

max_depth = [4, 5, 6]
estimators = [200, 300, 400, 500]
rate = [0.005, 0.008, 0.01]
eval_met = ['mape', 'rmse']

my_param_grid = ({'max_depth': max_depth,
                 'n_estimators': estimators,
                 'learning_rate': rate,
                 'eval_metric': eval_met})

# Cross-validation split for time-series data

my_cv = TimeSeriesSplit(n_splits=3).split(X_remainder)

In [20]:
# Instantiate the log reg grid search

gridsearch = GridSearchCV(xgb, param_grid=my_param_grid, cv=my_cv, verbose=1)

# Fit the log reg grid search

fitted_xgb = gridsearch.fit(X_remainder, y_remainder)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


In [21]:
# Best parameters

fitted_xgb.best_params_

{'eval_metric': 'mape',
 'learning_rate': 0.01,
 'max_depth': 4,
 'n_estimators': 500}

For the `learning rate` and `n_estimators`, the grid search still picks up the same parameters, however, the `maximum depth` is now 3. Since this is a middle value in our range, I can be confident that no bigger or smaller value will improve the model further. Now let's build the model using these parameters.

In [206]:
# Building the model with the best performing parameters

xgb = XGBRegressor(max_depth=4, n_estimators=500, learning_rate=0.01)
xgb.fit(X_remainder, y_remainder)

preds_xgb_train = xgb.predict(X_remainder).reshape(11690,1)
preds_xgb_test = xgb.predict(X_test).reshape(3286,1)

In [207]:
# Extracting MAPE scores

mape_train = round(np.mean(abs((y_remainder - preds_xgb_train) / y_remainder)) * 100, 2)
mape_test = round(np.mean(abs((y_test - preds_xgb_test) / y_test)) * 100, 2)

print(mape_train)
print(mape_test)

23.51
20.89


The model is not overfitting anymore, and the results improved a lot compared to the previous models. The test results are again slightly better than the train scores, however, as per the previous notebook, I'll assume this is because of the nature of the dataset.

Summary of the decision-tree based models' performance:

| Model  |  Train results |  Test results |
|---|---|---|
|  Baseline (mean prediction) | 99.39%  | 100.31%  |
|  Decision Tree (max depth = 1) | 36.52%  | 44.16%  |
|  XGBoost() | 15.49%  | 27.33%  |
|  XGBoost (max_depth=4, n_estimators=500, eta=0.01) | 23.51%  | 20.89%  |

***
### Brief summary

- The decision-tree model without optimization didn't do particularly well on the dataset, however, it was an improvement over the baseline model
- The first **XGBoost model** without hyperparameter optimization was **heavily overfitting** on the training data
- We managed to overcome the overfitting, and found the **best-performing XGBoost model through Grid search**
- However, it didn't do better than the linear models or the auto-regressive models, with a final **20.98% MAPE score over the test set**
- XGboost models are also quite difficult to interpret, which needs to be considered when picking the final model. Since they didn't overdo the previously models, I won't be going forward with this model for the final forecasting.

Summary table of the best-performing models:

| Model  |  Train results |  Test results |
|---|---|---|
|  Baseline (mean prediction) | 99.39%  | 100.31%  |
|  SARIMA(1,1,2)(1, 1, 1, 12) | 13.46%  | 12.18%  |
|  Linear regression | 24.36%  | 19.75%  |
|  XGBoost (max_depth=4, n_estimators=500, eta=0.01) | 23.51%  | 20.89%  |

***In the next section, I'll use the best-performing model (SARIMA) to forecast future AQI values and save the final model.***